Baseball Prediction: 5b - Adding Starting Pitching Features

    - Now that we have raw game-level data for each pitcher, we can derive features based on the starting pitchers to help our prediction model for individual games.


In [34]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

Overall Plan of Attack

    - For each starting pitcher we will load their raw data, create features for each game based on their previous performance, and then save the dataframe in a dictionary structure for easy lookup

    - Then we can iterate through our game-level dataframe, add in the features for each starting pitcher, and use those to improve our model

Once again, we will use C.C. Sabathia as our example candidate

In [35]:
df_cc = pd.read_csv('/Volumes/CharmedXi/beatVegas/SP_new/pitching_data_sabac001.csv')

In [36]:
df_cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   at_vs        561 non-null    object 
 1   Opponent     561 non-null    object 
 2   League       561 non-null    object 
 3   GS           561 non-null    int64  
 4   CG           561 non-null    int64  
 5   SHO          561 non-null    int64  
 6   GF           561 non-null    int64  
 7   SV           561 non-null    int64  
 8   IP           561 non-null    float64
 9   H            561 non-null    int64  
 10  BFP          561 non-null    int64  
 11  HR           561 non-null    int64  
 12  R            561 non-null    int64  
 13  ER           561 non-null    int64  
 14  BB           561 non-null    int64  
 15  IB           561 non-null    int64  
 16  SO           561 non-null    int64  
 17  SH           561 non-null    int64  
 18  SF           561 non-null    int64  
 19  WP      

In [37]:
df_cc.head(10)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,date,dblhead_num
0,VS,BAL,A,1,0,0,0,0,5.2,3,22,1,3,3,2,0,3,0,0,1,0,0,2,0,0,0,0,0,4.76,4- 8-2001,NaN
1,AT,DET,A,1,0,0,0,0,5.0,5,21,0,4,4,2,0,2,0,0,0,0,0,1,0,1,0,1,0,5.91,4-13-2001,NaN
2,AT,BAL,A,1,0,0,0,0,6.0,6,23,0,2,2,2,0,3,0,0,0,0,1,1,0,0,0,1,0,4.86,4-19-2001,NaN
3,VS,ANA,A,1,0,0,0,0,5.0,5,24,0,3,2,4,0,1,0,0,0,0,0,1,1,0,1,0,1,4.57,4-25-2001,NaN
4,AT,KC,A,1,0,0,0,0,5.0,4,19,1,1,1,1,0,2,0,0,0,0,0,0,0,1,0,1,0,4.05,5- 2-2001,NaN
5,VS,KC,A,1,0,0,0,0,5.2,5,23,0,1,1,1,0,5,0,1,1,1,0,1,0,0,0,1,0,3.62,5- 9-2001,NaN
6,AT,TEX,A,1,0,0,0,0,5.0,6,23,0,3,3,3,0,5,0,0,1,0,0,0,0,1,0,1,0,3.86,5-15-2001,NaN
7,AT,ANA,A,1,0,0,0,0,4.1,9,24,1,6,6,3,0,4,0,0,1,0,0,3,0,0,0,0,0,4.75,5-20-2001,NaN
8,VS,NY,A,1,0,0,0,0,4.2,7,25,2,5,5,4,1,3,0,0,0,0,1,2,0,0,0,0,1,5.24,5-26-2001,NaN
9,AT,NY,A,1,0,0,0,0,4.0,4,20,1,4,3,5,0,2,0,0,0,0,0,0,0,1,0,1,0,5.36,6- 1-2001,NaN


In [38]:
# Baseball statisticans report partial innings pitched as X.1 & X.2 where X is the integer number of innings pitched
# However, for calculation purposes, we need to tell the computer to recognize X.1 & X.2 as X & (one-third) and (two-thirds), respectively
# The following line of code does that for us.

df_cc['IP_real'] = df_cc['IP_real'] = (df_cc.IP - (df_cc.IP % 1)) + (df_cc.IP % 1) * (10/3)

In [39]:
df_cc.columns

Index(['at_vs', 'Opponent', 'League', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP', 'H',
       'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B', 'GDP', 'ROE', 'W', 'L', 'ERA', 'date', 'dblhead_num',
       'IP_real'],
      dtype='object')

Generating Features for a Starting Pitcher

    - Want to creature features based on a lookback across games for each pitcher

    - Need to aggregate statistics like innings pitched (IP), runs or earned runs given up, hits and walks given up, strikeouts, and so on

    - We will also need to decide how to handle pitchers early in their career when they have relatively few games on which to base their performance.

In [40]:
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

In [43]:
def load_and_process_pitch_df(p_id, filepath=''):
    fname = filepath+'pitching_data_'+p_id+'.csv'
    pitch_df = pd.read_csv(fname)
    
    # Convert date, fix dblhead_num to be 0,1,2
    pitch_df['date'] = (pd.to_datetime(pitch_df.date).astype(str).str.replace('-','')).astype(int)
    pitch_df.dblhead_num.fillna(0, inplace=True)
    pitch_df['dblhead_num'] = pitch_df['dblhead_num'].astype(int)
    
    # Convert IP to proper mathematical format
    pitch_df['IP_real'] = (pitch_df.IP - (pitch_df.IP % 1)) + (pitch_df.IP % 1) * (10/3)
    
    cols_to_agg = ['IP_real', 'H','BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B']
    winsizes = [3,14,30]
    for winsize in winsizes:
        for raw_col in cols_to_agg:
            new_colname = 'rollsum_'+raw_col+'_'+str(winsize)        
            pitch_df[new_colname] = roll_column(pitch_df, raw_col, winsize)

    
    er_per_ip_def = (5/9)
    h_bb_per_ip_def = 1.5
    h_bb_per_bf_def = .37
    so_per_bf_def = .2
    ip_per_game_def = 3
    bf_per_game_def = 12
    tb_bb_perc_def = .45
    fip_numer_per_ip_def = .124*13 + 1.5*3 - 2*.8
    fip_numer_per_bf_def = .03*13 + .37*3 - 2*.2
    for winsize in winsizes:
        hit_col = 'rollsum_H_'+str(winsize)
        bb_col = 'rollsum_BB_'+str(winsize)
        h_bb_col = 'H_BB_roll_'+str(winsize)
        double_col = 'rollsum_2B_'+str(winsize)
        triple_col = 'rollsum_3B_'+str(winsize)
        hr_col = 'rollsum_HR_'+str(winsize)
        xb_col = 'XB_roll_'+str(winsize)
        tb_col = 'TB_roll_'+str(winsize)
        so_col = 'rollsum_SO_'+str(winsize)
        so_mod_col = 'SO_mod_'+str(winsize)
        ip_col = 'rollsum_IP_real_'+str(winsize)
        ip_mod_col = 'IP_mod_'+str(winsize)
        er_col = 'rollsum_ER_'+str(winsize)
        er_mod_col = 'ER_mod_'+str(winsize)
        bf_col = 'rollsum_BFP_'+str(winsize)
        bf_mod_col = 'BF_mod_'+str(winsize)
        era_col = 'ERA_'+str(winsize)
        fip_col = 'FIP_'+str(winsize)
        fip_perc_col = 'FIP_perc_'+str(winsize)
        fip_numer_col = 'FIP_numer_'+str(winsize)
        fip_numer_mod_col = 'FIP_numer_mod_'+str(winsize)
        fip_numer_mod2_col = 'FIP_numer_mod2_'+str(winsize)
        whip_col = 'WHIP_'+str(winsize)
        so_perc_col = 'SO_perc_'+str(winsize)
        h_bb_perc_col = 'H_BB_perc_'+str(winsize)
        h_bb_mod_col = 'H_BB_mod_'+str(winsize)
        h_bb_mod2_col = 'H_BB_mod2_'+str(winsize)
        tb_bb_mod_col = 'TB_BB_mod_'+str(winsize)
        tb_bb_perc_col = 'TB_BB_perc_'+str(winsize)
        pitch_df[h_bb_col] = pitch_df[hit_col]+pitch_df[bb_col]
        pitch_df[xb_col] = pitch_df[double_col]+2*pitch_df[triple_col]+3*pitch_df[hr_col]
        pitch_df[tb_col] = pitch_df[hit_col]+pitch_df[xb_col]
        pitch_df[ip_mod_col] = np.maximum(pitch_df[ip_col], winsize*ip_per_game_def)
        pitch_df[bf_mod_col] = np.maximum(pitch_df[bf_col], winsize*bf_per_game_def)
        pitch_df[er_mod_col] = pitch_df[er_col] + er_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_col] = 13*pitch_df[hr_col] + 3*pitch_df[h_bb_col] -2*pitch_df[so_col]
        pitch_df[fip_numer_mod_col] = pitch_df[fip_numer_col] + fip_numer_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_mod2_col] = pitch_df[fip_numer_col] + fip_numer_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[h_bb_mod_col] = pitch_df[h_bb_col] + h_bb_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[h_bb_mod2_col] = pitch_df[h_bb_col] + h_bb_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[so_mod_col] = pitch_df[so_col] + so_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[tb_bb_mod_col] = (pitch_df[tb_col] + pitch_df[bb_col])+ tb_bb_perc_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[era_col] = (pitch_df[er_mod_col]/pitch_df[ip_mod_col])*9
        pitch_df[fip_col] = (pitch_df[fip_numer_mod_col]/pitch_df[ip_mod_col])
        pitch_df[fip_perc_col] = (pitch_df[fip_numer_mod_col]/pitch_df[bf_mod_col])
        pitch_df[whip_col] = pitch_df[h_bb_mod_col]/pitch_df[ip_mod_col]
        pitch_df[so_perc_col] = pitch_df[so_mod_col]/pitch_df[bf_mod_col]
        pitch_df[tb_bb_perc_col] = pitch_df[tb_bb_mod_col]/pitch_df[bf_mod_col]
        pitch_df[h_bb_perc_col] = pitch_df[h_bb_mod2_col]/pitch_df[bf_mod_col]
    pitch_df['date_dblhead'] = (pitch_df['date'].astype(str) + pitch_df['dblhead_num'].astype(str)).astype(int)
    pitch_df.set_index('date_dblhead', inplace=True)
    return(pitch_df)

In [44]:
df_cc = load_and_process_pitch_df('sabac001','SP_data/')

df_cc.head(15)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,date,dblhead_num,IP_real,rollsum_IP_real_3,rollsum_H_3,rollsum_BFP_3,rollsum_HR_3,rollsum_R_3,rollsum_ER_3,rollsum_BB_3,rollsum_IB_3,rollsum_SO_3,rollsum_SH_3,rollsum_SF_3,rollsum_WP_3,rollsum_HBP_3,rollsum_BK_3,rollsum_2B_3,rollsum_3B_3,rollsum_IP_real_14,rollsum_H_14,rollsum_BFP_14,rollsum_HR_14,rollsum_R_14,rollsum_ER_14,rollsum_BB_14,rollsum_IB_14,rollsum_SO_14,rollsum_SH_14,rollsum_SF_14,rollsum_WP_14,rollsum_HBP_14,rollsum_BK_14,rollsum_2B_14,rollsum_3B_14,rollsum_IP_real_30,rollsum_H_30,rollsum_BFP_30,rollsum_HR_30,rollsum_R_30,rollsum_ER_30,rollsum_BB_30,rollsum_IB_30,rollsum_SO_30,rollsum_SH_30,rollsum_SF_30,rollsum_WP_30,rollsum_HBP_30,rollsum_BK_30,rollsum_2B_30,rollsum_3B_30,H_BB_roll_3,XB_roll_3,TB_roll_3,IP_mod_3,BF_mod_3,ER_mod_3,FIP_numer_3,FIP_numer_mod_3,FIP_numer_mod2_3,H_BB_mod_3,H_BB_mod2_3,SO_mod_3,TB_BB_mod_3,ERA_3,FIP_3,FIP_perc_3,WHIP_3,SO_perc_3,TB_BB_perc_3,H_BB_perc_3,H_BB_roll_14,XB_roll_14,TB_roll_14,IP_mod_14,BF_mod_14,ER_mod_14,FIP_numer_14,FIP_numer_mod_14,FIP_numer_mod2_14,H_BB_mod_14,H_BB_mod2_14,SO_mod_14,TB_BB_mod_14,ERA_14,FIP_14,FIP_perc_14,WHIP_14,SO_perc_14,TB_BB_perc_14,H_BB_perc_14,H_BB_roll_30,XB_roll_30,TB_roll_30,IP_mod_30,BF_mod_30,ER_mod_30,FIP_numer_30,FIP_numer_mod_30,FIP_numer_mod2_30,H_BB_mod_30,H_BB_mod2_30,SO_mod_30,TB_BB_mod_30,ERA_30,FIP_30,FIP_perc_30,WHIP_30,SO_perc_30,TB_BB_perc_30,H_BB_perc_30
date_dblhead,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200104080,VS,BAL,A,1,0,0,0,0,5.2,3,22,1,3,3,2,0,3,0,0,1,0,0,2,0,0,0,0,0,4.76,20010408,0,5.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,36.0,5.000000,0.0,40.608,39.6,13.5,13.32,7.2,16.2,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000,0.0,0.0,0.0,42.000000,168.0,23.333333,0.0,189.504,184.8,63.0,62.16,33.6,75.60,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000,0.0,0.0,0.0,90.0,360.0,50.000000,0.0,406.080,396.0,135.0,133.20,72.0,162.00,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000
200104130,AT,DET,A,1,0,0,0,0,5.0,5,21,0,4,4,2,0,2,0,0,0,0,0,1,0,1,0,1,0,5.91,20010413,0,5.000000,5.666667,3.0,22.0,1.0,3.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,5.666667,3.0,22.0,1.0,3.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,5.666667,3.0,22.0,1.0,3.0,3.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,5.0,8.0,9.000000,36.0,4.851852,22.0,37.040,37.4,10.0,10.18,5.8,16.3,4.851852,4.115556,1.028889,1.111111,0.161111,0.452778,0.282778,5.0,5.0,8.0,42.000000,168.0,23.185185,22.0,185.936,182.6,59.5,59.02,32.2,75.70,4.968254,4.427048,1.106762,1.416667,0.191667,0.450595,0.351310,5.0,5.0,8.0,90.0,360.0,49.851852,22.0,402.512,393.8,131.5,130.06,70.6,162.10,4.985185,4.472356,1.118089,1.461111,0.196111,0.450278,0.361278
200104190,AT,BAL,A,1,0,0,0,0,6.0,6,23,0,2,2,2,0,3,0,0,0,0,1,1,0,0,0,1,0,4.86,20010419,0,6.000000,10.666667,8.0,43.0,1.0,7.0,7.0,4.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,10.666667,8.0,43.0,1.0,7.0,7.0,4.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,10.666667,8.0,43.0,1.0,7.0,7.0,4.0,0.0,5.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,12.0,6.0,14.0,10.666667,43.0,7.000000,39.0,39.000,39.0,12.0,12.00,5.0,18.0,5.906250,3.656250,0.906977,1.125000,0.116279,0.418605,0.279070,12.0,6.0,14.0,42.000000,168.0,24.407407,39.0,180.376,176.5,59.0,58.25,30.0,74.25,5.230159,4.294667,1.073667,1.404762,0.178571,0.441964,0.346726,12.0,6.0,14.0,90.0,360.0,51.074074,39.0,396.952,387.7,131.0,129.29,68.4,160.65,5.107407,4.410578,1.102644,1.455556,0.190000,0.446250,0.359139
200104250,VS,ANA,A,1,0,0,0,0,5.0,5,24,0,3,2,4,0,1,0,0,0,0,0,1,1,0,1,0,1,4.57,20010425,0,5.000000,16.666667,14.0,66.0,1.0,9.0,9.0,6.0,0.0,8.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,16.666667,14.0,66.0,1.0,

LOAD IN GAME LEVEL DATA

In [45]:
df = pd.read_csv('df_bp3.csv', low_memory=False)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56771 entries, 0 to 56770
Columns: 199 entries, date to over_under_result
dtypes: float64(34), int64(87), object(78)
memory usage: 86.2+ MB


In [47]:
start_pitchers_h = df.pitcher_start_id_h.unique()
start_pitchers_v = df.pitcher_start_id_v.unique()
start_pitchers_all = np.union1d(start_pitchers_h.astype(str), start_pitchers_v.astype(str))

In [48]:
pitcher_data_dict = {}
for i, p_id in enumerate(start_pitchers_all):
    if i % 250 == 0:
        print(i)
    try:  
        pitcher_data_dict[p_id] = load_and_process_pitch_df(p_id,'SP_new/')
    except:
       pass

0
250
500
750
1000
1250
1500
1750
2000


In [49]:
raw_cols_to_add = ['GS', 'IP',
       'H', 'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP',
       'HBP', 'BK', '2B', '3B', 'IP_real', 'rollsum_IP_real_3', 'rollsum_H_3',
       'rollsum_BFP_3', 'rollsum_HR_3', 'rollsum_R_3', 'rollsum_ER_3',
       'rollsum_BB_3', 'rollsum_IB_3', 'rollsum_SO_3', 'rollsum_SH_3',
       'rollsum_SF_3', 'rollsum_WP_3', 'rollsum_HBP_3',
       'rollsum_BK_3', 'rollsum_2B_3', 'rollsum_3B_3',
       'rollsum_IP_real_14', 'rollsum_H_14', 'rollsum_BFP_14',
       'rollsum_HR_14', 'rollsum_R_14', 'rollsum_ER_14', 'rollsum_BB_14',
       'rollsum_IB_14', 'rollsum_SO_14', 'rollsum_SH_14', 'rollsum_SF_14',
       'rollsum_WP_14', 'rollsum_HBP_14', 'rollsum_BK_14',
       'rollsum_2B_14', 'rollsum_3B_14', 'rollsum_IP_real_30',
       'rollsum_H_30', 'rollsum_BFP_30', 'rollsum_HR_30', 'rollsum_R_30',
       'rollsum_ER_30', 'rollsum_BB_30', 'rollsum_IB_30', 'rollsum_SO_30',
       'rollsum_SH_30', 'rollsum_SF_30', 'rollsum_WP_30',
       'rollsum_HBP_30', 'rollsum_BK_30', 'rollsum_2B_30',
       'rollsum_3B_30', 'H_BB_roll_3', 'XB_roll_3', 'TB_roll_3',
       'IP_mod_3', 'BF_mod_3', 'ER_mod_3', 'FIP_numer_3',
       'FIP_numer_mod_3', 'FIP_numer_mod2_3', 'H_BB_mod_3',
       'H_BB_mod2_3', 'SO_mod_3', 'TB_BB_mod_3', 'ERA_3', 'FIP_3',
       'FIP_perc_3', 'WHIP_3', 'SO_perc_3', 'TB_BB_perc_3',
       'H_BB_perc_3', 'H_BB_roll_14', 'XB_roll_14', 'TB_roll_14',
       'IP_mod_14', 'BF_mod_14', 'ER_mod_14', 'FIP_numer_14',
       'FIP_numer_mod_14', 'FIP_numer_mod2_14', 'H_BB_mod_14',
       'H_BB_mod2_14', 'SO_mod_14', 'TB_BB_mod_14', 'ERA_14', 'FIP_14',
       'FIP_perc_14', 'WHIP_14', 'SO_perc_14', 'TB_BB_perc_14',
       'H_BB_perc_14', 'H_BB_roll_30', 'XB_roll_30', 'TB_roll_30',
       'IP_mod_30', 'BF_mod_30', 'ER_mod_30', 'FIP_numer_30',
       'FIP_numer_mod_30', 'FIP_numer_mod2_30', 'H_BB_mod_30',
       'H_BB_mod2_30', 'SO_mod_30', 'TB_BB_mod_30', 'ERA_30', 'FIP_30',
       'FIP_perc_30', 'WHIP_30', 'SO_perc_30', 'TB_BB_perc_30',
       'H_BB_perc_30']
cols_to_add = ['Strt_'+col+suff for col in raw_cols_to_add for suff in ['_h','_v']]

col_add_dict = {col:np.zeros(df.shape[0]) for col in cols_to_add}

In [50]:
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    sp_id_v = row['pitcher_start_id_v']
    sp_id_h = row['pitcher_start_id_h']
    date_dblhead = row['date_dblhead']
    if sp_id_v in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_v]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_v'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_v} date {date_dblhead}')
    if sp_id_h in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_h]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_h'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_h} date {date_dblhead}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000


In [51]:
for col in cols_to_add:
    df[col] = col_add_dict[col]

In [52]:
df.sample(5)


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_3_h,Strt_rollsum_IP_real_3_v,Strt_rollsum_H_3_h,Strt_rollsum_H_3_v,Strt_rollsum_BFP_3_h,Strt_rollsum_BFP_3_v,Strt_rollsum_HR_3_h,Strt_rollsum_HR_3_v,Strt_rollsum_R_3_h,Strt_rollsum_R_3_v,Strt_rollsum_ER_3_h,Strt_rollsum_ER_3_v,Strt_rollsum_BB_3_h,Strt_rollsum_BB_3_v,Strt_rollsum_IB_3_h,Strt_rollsum_IB_3_v,Strt_rollsum_SO_3_h,Strt_rollsum_SO_3_v,Strt_rollsum_SH_3_h,Strt_rollsum_SH_3_v,Strt_rollsum_SF_3_h,Strt_rollsum_SF_3_v,Strt_rollsum_WP_3_h,Strt_rollsum_WP_3_v,Strt_rollsum_HBP_3_h,Strt_rollsum_HBP_3_v,Strt_rollsum_BK_3_h,Strt_rollsum_BK_3_v,Strt_rollsum_2B_3_h,Strt_rollsum_2B_3_v,Strt_rollsum_3B_3_h,Strt_rollsum_3B_3_v,Strt_rollsum_IP_real_14_h,Strt_rollsum_IP_real_14_v,Strt_rollsum_H_14_h,Strt_rollsum_H_14_v,Strt_rollsum_BFP_14_h,Strt_rollsum_BFP_14_v,Strt_rollsum_HR_14_h,Strt_rollsum_HR_14_v,Strt_rollsum_R_14_h,Strt_rollsum_R_14_v,Strt_rollsum_ER_14_h,Strt_rollsum_ER_14_v,Strt_rollsum_BB_14_h,Strt_rollsum_BB_14_v,Strt_rollsum_IB_14_h,Strt_rollsum_IB_14_v,Strt_rollsum_SO_14_h,Strt_rollsum_SO_14_v,Strt_rollsum_SH_14_h,Strt_rollsum_SH_14_v,Strt_rollsum_SF_14_h,Strt_rollsum_SF_14_v,Strt_rollsum_WP_14_h,Strt_rollsum_WP_14_v,Strt_rollsum_HBP_14_h,Strt_rollsum_HBP_14_v,Strt_rollsum_BK_14_h,Strt_rollsum_BK_14_v,Strt_rollsum_2B_14_h,Strt_rollsum_2B_14_v,Strt_rollsum_3B_14_h,Strt_rollsum_3B_14_v,Strt_rollsum_IP_real_30_h,Strt_rollsum_IP_real_30_v,Strt_rollsum_H_30_h,Strt_rollsum_H_30_v,Strt_rollsum_BFP_30_h,Strt_rollsum_BFP_30_v,Strt_rollsum_HR_30_h,Strt_rollsum_HR_30_v,Strt_rollsum_R_30_h,Strt_rollsum_R_30_v,

In [53]:
df.shape


(56771, 451)

In [54]:
(df.Strt_IP_mod_3_h==0).sum()


0

In [55]:
(df.Strt_IP_mod_3_v==0).sum()


0

In [ ]:
df.loc[df.Strt_IP_mod_3_v==0]


In [56]:
df.loc[df.Strt_IP_mod_3_h==0]


,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_3_h,Strt_rollsum_IP_real_3_v,Strt_rollsum_H_3_h,Strt_rollsum_H_3_v,Strt_rollsum_BFP_3_h,Strt_rollsum_BFP_3_v,Strt_rollsum_HR_3_h,Strt_rollsum_HR_3_v,Strt_rollsum_R_3_h,Strt_rollsum_R_3_v,Strt_rollsum_ER_3_h,Strt_rollsum_ER_3_v,Strt_rollsum_BB_3_h,Strt_rollsum_BB_3_v,Strt_rollsum_IB_3_h,Strt_rollsum_IB_3_v,Strt_rollsum_SO_3_h,Strt_rollsum_SO_3_v,Strt_rollsum_SH_3_h,Strt_rollsum_SH_3_v,Strt_rollsum_SF_3_h,Strt_rollsum_SF_3_v,Strt_rollsum_WP_3_h,Strt_rollsum_WP_3_v,Strt_rollsum_HBP_3_h,Strt_rollsum_HBP_3_v,Strt_rollsum_BK_3_h,Strt_rollsum_BK_3_v,Strt_rollsum_2B_3_h,Strt_rollsum_2B_3_v,Strt_rollsum_3B_3_h,Strt_rollsum_3B_3_v,Strt_rollsum_IP_real_14_h,Strt_rollsum_IP_real_14_v,Strt_rollsum_H_14_h,Strt_rollsum_H_14_v,Strt_rollsum_BFP_14_h,Strt_rollsum_BFP_14_v,Strt_rollsum_HR_14_h,Strt_rollsum_HR_14_v,Strt_rollsum_R_14_h,Strt_rollsum_R_14_v,Strt_rollsum_ER_14_h,Strt_rollsum_ER_14_v,Strt_rollsum_BB_14_h,Strt_rollsum_BB_14_v,Strt_rollsum_IB_14_h,Strt_rollsum_IB_14_v,Strt_rollsum_SO_14_h,Strt_rollsum_SO_14_v,Strt_rollsum_SH_14_h,Strt_rollsum_SH_14_v,Strt_rollsum_SF_14_h,Strt_rollsum_SF_14_v,Strt_rollsum_WP_14_h,Strt_rollsum_WP_14_v,Strt_rollsum_HBP_14_h,Strt_rollsum_HBP_14_v,Strt_rollsum_BK_14_h,Strt_rollsum_BK_14_v,Strt_rollsum_2B_14_h,Strt_rollsum_2B_14_v,Strt_rollsum_3B_14_h,Strt_rollsum_3B_14_v,Strt_rollsum_IP_real_30_h,Strt_rollsum_IP_real_30_v,Strt_rollsum_H_30_h,Strt_rollsum_H_30_v,Strt_rollsum_BFP_30_h,Strt_rollsum_BFP_30_v,Strt_rollsum_HR_30_h,Strt_rollsum_HR_30_v,Strt_rollsum_R_30_h,Strt_rollsum_R_30_v,

In [57]:
df.drop(df.index[df.Strt_IP_mod_3_v==0],inplace=True)
df.drop(df.index[df.Strt_IP_mod_3_h==0],inplace=True)
df.shape


(56771, 451)

In [58]:
df.reset_index(drop=True, inplace=True)

In [59]:
df.to_csv('df_bp5.csv', index=False)